In [1]:
import os, re, time

In [2]:
import lxml.etree as ET

In [3]:
#get a list of the wordfiles which should be sent to http://www.tei-c.org/oxgarage/
wordfiles = os.listdir("./data/word")

In [4]:
# send each wordfile to http://www.tei-c.org/oxgarage/ where it get
# transformed to XML/TEI and saved as data/fromOxgarage/{filename}.xml
for file in wordfiles:
    cmd = ("curl -o data/fromOxgarage/{0} -F upload=@data/word/{1} http://oxgarage.oucs.ox.ac.uk:8080/ege-webservice/Conversions/docx%3Aapplication%3Avnd.openxmlformats-officedocument.wordprocessingml.document/TEI%3Atext%3Axml/conversion?properties=%3Cconversions%3E%3Cconversion%20index=%220%22%3E%3Cproperty%20id=%22oxgarage.getImages%22%3Etrue%3C/property%3E%3Cproperty%20id=%22oxgarage.getOnlineImages%22%3Etrue%3C/property%3E%3Cproperty%20id=%22oxgarage.lang%22%3Een%3C/property%3E%3Cproperty%20id=%22oxgarage.textOnly%22%3Efalse%3C/property%3E%3Cproperty%20id=%22pl.psnc.dl.ege.tei.profileNames%22%3Edefault%3C/property%3E%3C/conversion%3E%3C/conversions%3E".format(((os.path.splitext(file)[0])+".xml"), file))
    os.system(cmd)
    time.sleep(1)

In [5]:
#load the xsl-file and create a xsl-transform object
xslt = ET.parse("oxgarageToMyTEI.xsl")
transform = ET.XSLT(xslt)

In [6]:
#iterate over data/fromOxgarage/ -directory, transform every XML-File and save the result in another directory
for file in os.listdir("./data/fromOxgarage"):
    if  file.endswith(".xml"):
        dom = ET.parse("./data/fromOxgarage/"+file)
        newdom = transform(dom)
        newdom.write(str("./data/myTEI/"+file))

In [7]:
#iterate over each myTEI file and perform some regex-operations
for file in os.listdir("./data/myTEI"):#iterate over files in the directory
    data = open("./data/myTEI/"+file, "r", encoding="utf-8")# open each file
    text = data.read()# read the file and save content as string to "text"
    data.close()# close the file
    replace = str('TEI xmlns="http://www.tei-c.org/ns/1.0"'+ ' xml:id="'+file+'"') # add the file name as xml:id in the root element
    text = re.sub(r'TEI xmlns="http://www.tei-c.org/ns/1.0"', replace, text) # add the file name as xml:id in the root element
    text = re.sub(r"<p>\[Bl\.\s*(?P<zahl>\d+)\s*(?P<rectoverso>[r|v])\]\s*</p>",'<pb n="\g<zahl>\g<rectoverso>"/>',text)
    matches = re.findall(r'<rs type="\w+" ref="#\w+_\w+">', text)
    for match in matches:
        text = re.sub(match, match.lower(), text)
    matches = re.findall(r'<rs type="\w+" ref="#\w+">', text)
    for match in matches:
        text = re.sub(match, match.lower(), text)
    text = re.sub(r'(?P<all><p>(?P<match>\w*\s*\w*)</p><table rend="rules">)',
           '\g<all><head type="added">\g<match></head>', text)#add table headers
    output_text = open("./data/myTEI/"+file, "w")
    print(text, file=output_text)
    output_text.close()